<a href="https://colab.research.google.com/github/aman9213/demo/blob/master/DL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader 
import pathlib

In [2]:
from google.colab import files

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/nature_12K.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/nature') #Extracts the files into the /nature folder
zip_ref.close()

In [5]:
transformer=transforms.Compose([transforms.Resize((200,200)),transforms.RandomHorizontalFlip(),transforms.RandomRotation((-90,90)),transforms.RandomAffine(degrees=0,scale=(0.3,0.3)),transforms.ToTensor(),transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])
                            


In [6]:
batch=256
#dataloader
train_path='/nature/inaturalist_12K/train'
test_path='/nature/inaturalist_12K/val'
train_loader=DataLoader(torchvision.datasets.ImageFolder(train_path,transform=transformer),
                        batch_size=batch,shuffle=True)
test_loader=DataLoader(torchvision.datasets.ImageFolder(test_path,transform=transformer),
                        batch_size=batch,shuffle=True)


In [7]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [8]:
classes=(classes[1:])

In [9]:
print(classes)

['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [20]:
K=32
F=3
batch_norm=True
activation='Relu'
nodes=512
class ConvNet(nn.Module):
  def __init__(self,num_class=10):
    super(ConvNet,self).__init__()
    self.conv1=nn.Conv2d(in_channels=3,out_channels=K,kernel_size=F,stride=1,padding=(F-1)/2) #its output size will be=(batch=256,K,200,200)
    if batch_norm:
      self.bn1=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.relu1=nn.ReLU()
    self.maxpool1=nn.MaxPool2d(kernel_size=2)## shape=(256,K,100,100)
    #layer 2
    self.conv2=nn.Conv2d(in_channels=K,out_channels=K,kernel_size=F,stride=1,padding=(F-1)/2) #its output size will be=(batch=256,K,200,200)
    if batch_norm:
      self.bn2=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.relu2=nn.ReLU()
    self.maxpool2=nn.MaxPool2d(kernel_size=2)
    ##shape=(256,K,50,50)
    # layer 3
    self.conv3=nn.Conv2d(in_channels=K,out_channels=K,kernel_size=F,stride=1,padding=(F-1)/2) #its output size will be=(batch=256,K,200,200)
    if batch_norm:
      self.bn3=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.relu3=nn.ReLU()
    self.maxpool3=nn.MaxPool2d(kernel_size=2)
    ##shape=(256,K,25,25)
    #layer 4
    self.conv4=nn.Conv2d(in_channels=K,out_channels=K,kernel_size=F,stride=1,padding=(F-1)/2) #its output size will be=(batch=256,K,200,200)
    if batch_norm:
      self.bn4=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.relu4=nn.ReLU()
    self.maxpool4=nn.MaxPool2d(kernel_size=2)
    ## shape=(256,K,12,12)
    #layer 5
    self.conv5=nn.Conv2d(in_channels=K,out_channels=K,kernel_size=F,stride=1,padding=(F-1)/2) #its output size will be=(batch=256,K,200,200)
    if batch_norm:
      self.bn5=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.relu5=nn.ReLU()
    self.maxpool5=nn.MaxPool2d(kernel_size=2)
    ## shape(256,K,6,6)
    #print(len(self.maxpool5))
     ## shape(256,K,)
    ## FC layer
    self.fc1=nn.Linear(in_features=6*6*K,out_features=nodes)
    self.fc2=nn.Linear(in_features=nodes,out_features=10)


net=ConvNet()
print(net)

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1.0, 1.0))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1.0, 1.0))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1.0, 1.0))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1.0, 1.0))
  (bn4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_sta